In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re
import json

In [2]:
"""
Physics Scene Compiler and Test Scene Factory for Planck.js (V3)

This module provides two main classes:
1. PhysicsSceneCompiler: Acts as a translator from a high-level MCP source
   to the low-level JSON format required by the Planck.js frontend.
2. SceneFactory: A utility class that provides static methods to generate
   pre-defined, complex MCP scene dictionaries for testing purposes.
"""

import numpy as np
from typing import Dict, Any, Optional, Tuple, List

# ==============================================================================
#  PHYSICS SCENE COMPILER (UNCHANGED)
# ==============================================================================

class PhysicsSceneCompiler:
    """
    A generic physics scene compiler that converts high-level MCP data into the
    low-level JSON format usable by Planck.js.
    ... (rest of the class is unchanged) ...
    """
    def __init__(self, gravity_g: float = 9.8):
        self.G = gravity_g

    def _calculate_density(self, obj: Dict[str, Any]) -> float:
        mass = obj.get("mass", 1.0)
        shape = obj.get("shape")
        if shape == "box":
            size = obj.get("size", {"width": 1.0, "height": 1.0})
            area = size["width"] * size["height"]
        elif shape == "circle":
            radius = obj.get("radius", 1.0)
            area = np.pi * radius ** 2
        else:
            area = 1.0
        if area == 0:
            return 1.0
        return mass / area

    def _prepare_pulley_joint(self, joint_mcp: Dict[str, Any], objects_map: Dict[str, Any]) -> Dict[str, Any]:
        obj_a_id = joint_mcp["object_a_id"]
        obj_b_id = joint_mcp["object_b_id"]
        if obj_a_id not in objects_map or obj_b_id not in objects_map:
            raise ValueError(f"Object ID referenced for PulleyJoint not found: {obj_a_id} or {obj_b_id}")
        pos_a = objects_map[obj_a_id]["position"]
        pos_b = objects_map[obj_b_id]["position"]
        pulley_pos = joint_mcp["pulley_anchor_pos"]
        length_a = np.sqrt((pos_a['x'] - pulley_pos['x'])**2 + (pos_a['y'] - pulley_pos['y'])**2)
        length_b = np.sqrt((pos_b['x'] - pulley_pos['x'])**2 + (pos_b['y'] - pulley_pos['y'])**2)
        return {
            "type": "PulleyJoint", "object_a_id": obj_a_id, "object_b_id": obj_b_id,
            "ground_anchor_a": pulley_pos, "ground_anchor_b": pulley_pos,
            "local_anchor_a": {"x": 0, "y": 0}, "local_anchor_b": {"x": 0, "y": 0},
            "length_a": length_a, "length_b": length_b,
            "ratio": joint_mcp.get("ratio", 1.0)
        }

    def compile_scene(self, mcp_data: Dict[str, Any]) -> Tuple[Optional[Dict[str, Any]], Optional[str]]:
        try:
            final_scene = {
                "world": mcp_data.get("world", {"gravity": {"x": 0, "y": -self.G}}),
                "objects": [], "joints": []
            }
            objects_map = {obj['id']: obj for obj in mcp_data.get("objects", [])}
            for obj_mcp in mcp_data.get("objects", []):
                planck_obj = obj_mcp.copy()
                if planck_obj.get("type") == "dynamic":
                    if "mass" in planck_obj and "density" not in planck_obj:
                        planck_obj["density"] = self._calculate_density(planck_obj)
                    elif "density" not in planck_obj:
                        planck_obj["density"] = 1.0
                final_scene["objects"].append(planck_obj)
            for joint_mcp in mcp_data.get("joints", []):
                joint_type = joint_mcp.get("type")
                if joint_type == "PulleyJoint":
                    planck_joint = self._prepare_pulley_joint(joint_mcp, objects_map)
                    final_scene["joints"].append(planck_joint)
                else:
                    final_scene["joints"].append(joint_mcp)
            return {"planck_scene": final_scene}, None
        except (KeyError, ValueError) as e:
            return None, f"Error processing MCP data: Invalid or missing key. Details: {str(e)}"
        except Exception as e:
            return None, f"An unexpected error occurred in the scene compiler: {str(e)}"

In [3]:
# ==============================================================================
#  NEW SCENE FACTORY CLASS
# ==============================================================================

class SceneFactory:
    """
    A factory for creating pre-defined, hard-coded physics scenes for testing.

    Each method is a static method that returns a complete MCP dictionary,
    ready to be used by the PhysicsSceneCompiler.
    """

    @staticmethod
    def create_domino_topple_scene() -> Dict[str, Any]:
        """Generates a scene with a line of dominoes and a ball to knock them over."""
        objects: List[Dict[str, Any]] = [
            # The ground
            {"id": "ground", "type": "static", "shape": "box", "size": {"width": 100, "height": 2}, "position": {"x": 50, "y": 1}},
            # The ball that starts the chain reaction
            {"id": "starter_ball", "type": "dynamic", "shape": "circle", "radius": 1, "mass": 5,
             "position": {"x": 5, "y": 2.5}, "linearVelocity": {"x": 25, "y": 0}, "restitution": 0.5}
        ]
        # Generate 15 dominoes
        for i in range(15):
            domino = {
                "id": f"domino_{i}",
                "type": "dynamic",
                "shape": "box",
                "size": {"width": 0.4, "height": 4},
                "position": {"x": 12 + i * 2.5, "y": 3},
                "mass": 1,
                "friction": 0.8
            }
            objects.append(domino)
        return {"world": {"gravity": {"x": 0, "y": -10}}, "objects": objects, "joints": []}

    @staticmethod
    def create_complex_pulley_scene() -> Dict[str, Any]:
        """Generates a scene with 3 bodies connected by 2 pulley joints."""
        objects = [
            {"id": "ground", "type": "static", "shape": "box", "size": {"width": 100, "height": 2}, "position": {"x": 50, "y": 1}},
            {"id": "box_A", "type": "dynamic", "shape": "box", "mass": 12, "size": {"width": 3, "height": 3}, "position": {"x": 20, "y": 15}},
            {"id": "box_B", "type": "dynamic", "shape": "box", "mass": 5, "size": {"width": 2, "height": 2}, "position": {"x": 40, "y": 30}},
            {"id": "box_C", "type": "dynamic", "shape": "box", "mass": 12, "size": {"width": 3, "height": 3}, "position": {"x": 60, "y": 15}},
        ]
        joints = [
            # First pulley connects A and B
            {"type": "PulleyJoint", "object_a_id": "box_A", "object_b_id": "box_B", "pulley_anchor_pos": {"x": 30, "y": 40}},
            # Second pulley connects B and C
            {"type": "PulleyJoint", "object_a_id": "box_B", "object_b_id": "box_C", "pulley_anchor_pos": {"x": 50, "y": 40}},
        ]
        return {"world": {"gravity": {"x": 0, "y": -9.8}}, "objects": objects, "joints": joints}

    @staticmethod
    def create_object_stack_scene() -> Dict[str, Any]:
        """Generates a scene with a stack of boxes of varying sizes to test stability."""
        objects: List[Dict[str, Any]] = [
            {"id": "ground", "type": "static", "shape": "box", "size": {"width": 100, "height": 2}, "position": {"x": 50, "y": 1}},
        ]
        # Generate a stack of 10 boxes
        box_width = 6
        y_pos = 3.5 # Start y-position for the bottom box's center
        for i in range(10):
            box = {
                "id": f"stack_box_{i}",
                "type": "dynamic",
                "shape": "box",
                "size": {"width": box_width, "height": 1},
                "position": {"x": 40, "y": y_pos},
                "mass": 10,
                "friction": 0.7,
                "restitution": 0.1
            }
            objects.append(box)
            # Next box is slightly smaller and placed on top
            y_pos += 1
            box_width *= 0.9
        return {"world": {"gravity": {"x": 0, "y": -10}}, "objects": objects, "joints": []}

    @staticmethod
    def create_newtons_cradle_scene() -> Dict[str, Any]:
        """Generates a scene resembling Newton's cradle to test elastic collisions."""
        objects: List[Dict[str, Any]] = [
            {"id": "ground", "type": "static", "shape": "box", "size": {"width": 100, "height": 2}, "position": {"x": 50, "y": 1}},
        ]
        num_balls = 6
        radius = 1.5
        y_pos = 10
        # Create a line of touching balls
        for i in range(num_balls):
            ball = {
                "id": f"ball_{i}",
                "type": "dynamic",
                "shape": "circle",
                "radius": radius,
                "position": {"x": 30 + i * (radius * 2), "y": y_pos},
                "mass": 1,
                "friction": 0.1,
                "restitution": 1.0, # Perfectly elastic
                "linearDamping": 0.05
            }
            objects.append(ball)

        # Add one ball to the side with an initial velocity to start the process
        starter_ball = {
            "id": "starter_ball",
            "type": "dynamic",
            "shape": "circle",
            "radius": radius,
            "position": {"x": 30 - (radius * 4), "y": y_pos},
            "mass": 1,
            "friction": 0.1,
            "restitution": 1.0,
            "linearVelocity": {"x": 30, "y": 0}
        }
        objects.append(starter_ball)
        return {"world": {"gravity": {"x": 0, "y": 0}}, "objects": objects, "joints": []} # No gravity for a clearer effect

In [4]:
# ==============================================================================
#  EXAMPLE USAGE
# ==============================================================================

# 1. Initialize the compiler
compiler = PhysicsSceneCompiler()

# 2. Get a pre-defined scene from the factory
# You can choose which scene to test by uncommenting it.

# print("--- Testing: Domino Topple Scene ---")
# mcp_payload = SceneFactory.create_domino_topple_scene()

print("\n--- Testing: Complex Pulley Scene ---")
mcp_payload = SceneFactory.create_complex_pulley_scene()

# print("\n--- Testing: Object Stack Scene ---")
# mcp_payload = SceneFactory.create_object_stack_scene()

# print("\n--- Testing: Newton's Cradle Scene ---")
# mcp_payload = SceneFactory.create_newtons_cradle_scene()


# 3. Compile the scene using the compiler
scene_data, error_message = compiler.compile_scene(mcp_payload)

# 4. Print the result
if error_message:
    print(f"Compilation Failed: {error_message}")
else:
    import json
    print("Compilation Successful! Generated Planck.js scene data:")
    print(json.dumps(scene_data, indent=2))


--- Testing: Complex Pulley Scene ---
Compilation Successful! Generated Planck.js scene data:
{
  "planck_scene": {
    "world": {
      "gravity": {
        "x": 0,
        "y": -9.8
      }
    },
    "objects": [
      {
        "id": "ground",
        "type": "static",
        "shape": "box",
        "size": {
          "width": 100,
          "height": 2
        },
        "position": {
          "x": 50,
          "y": 1
        }
      },
      {
        "id": "box_A",
        "type": "dynamic",
        "shape": "box",
        "mass": 12,
        "size": {
          "width": 3,
          "height": 3
        },
        "position": {
          "x": 20,
          "y": 15
        },
        "density": 1.3333333333333333
      },
      {
        "id": "box_B",
        "type": "dynamic",
        "shape": "box",
        "mass": 5,
        "size": {
          "width": 2,
          "height": 2
        },
        "position": {
          "x": 40,
          "y": 30
        },
        "dens